In [ ]:
!pip install higher


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import higher

import sys
sys.path.append('helpers')



## Meta Learning for Regression Using Closed Form Min-norm Solution

As described in the setup to the theory portion of this problem, we wish to learn a good set of feature weights for regression on tasks sampled from a distribution $\mathcal{D}_T$. We define this distribution in the cell below with several entries in `params_dict`. Specifically, the true feature indices are set by `k_idx` to be $\{5,6,7,8,9,10,11\}$ by default and the true coefficients of the features are generated with i.i.d $U[-1, 1]$ samples then normalized to have total length 1. The total number of features is set by `d` in the params dictionary. The $x$ sample spacing for the inner loop of training is determined by `x_type`, while the meta update is always performed using uniform random spaced samples (otherwise we can't differentiate between aliased features). For now we will use only uniform random samples. This is the most natural version since we require uniform random samples for the meta update and there's no reason we would have a different spacing for the inner training loop.

One significant difference between MAML as described in the original paper and as we use it in the notebook is that we use the closed form min-norm solution for regression instead of gradient descent. Fortunately for us, PyTorch is able to backpropagate gradients through matrix inversion which means that we can update our feature weights even though we use the min-norm least squares solution rather than gradient descent steps to choose our coefficients. In later parts we use gradient descent for the inner loop instead.




### (e)  Run the next two cells and answer the following questions:

1. Based on the plot of regression test loss vs n_train_post, how does the meta-learned feature weights perform as compared to the case where all feature weights are 1? How do the meta-learned feature weights perform as compared to the oracle (that performs regression only using the features present in the data)?

2. By looking at the evolution of the feature-weights with time as we perform meta-learning can you justify the improvement in performance?



In [ ]:
from utils_reg import meta_learning_reg_closed_form

def get_params_dict_reg():

    params_dict = {
        #Parameters
    'seed':7,
    'n_train_inner':32, #Number of training samples for training inner task
    'n_train_meta':64, #Number of training samples for updating the meta parameter

    'n_train_post':32, #Number of training samples used after meta training has been done to learn the weights
    'n_test_post':1000, #Number of samples used for plotting and evaluating test performance after meta training has been done

    # 'x_type':'uniform_random', #sampling time for inner and post training tasks
    # 'x_type':'grid', #sampling time for training tasks

    'd':501, #Number of features
    'phi_type':'fourier', #Feature type
     'noise_std':1e-1, #standard deviation of awgn noise added during training
    'optimizer_type':'SGD', #Can be either SGD or Adam
    'k_idx':np.arange(5,11), #Frequency range present in tasks during meta training
    'stepsize_meta':1e-2, #Stepsize used for meta updates

     'num_inner_tasks':5, #Number of inner tasks for each meta update
    'num_tasks_test':10, #Number of tasks to test on
    'num_stats': 10, #Determines how often we collect stats

    'num_iterations':100, #Iterations for training meta parameter
        'num_n_train_post_range':40, #How many points do we use to generative test loss vs n_train_post curve

    }
    return params_dict



In [ ]:
params_dict = get_params_dict_reg()
x_type = 'uniform_random'
cparams_dict = params_dict.copy()
cparams_dict['x_type'] = x_type
meta_learned_feature_weights_reg_closed_form, data_dict_reg = meta_learning_reg_closed_form(cparams_dict)


#### Output figures explained:

For each logged iteration, we visualize two rows of two subfigures, four in total.

On the first row, we plot (with log scale) the test MSE loss w.r.t. the number of datapoints used for linear regression after meta training is done. The green curve plots test loss from the oracle, i.e. using only the features present in the true signal; the blue curve uses feature weights learned from doing meta training. To see how the blue curve chages as we do more iterations of meta training, we use the orange curve to mark the initial iteration-0 location of the blue curve. For each curve the solid line corresponds to the average test loss over 10 tasks, while the shaded band corresponds to the range between the 10th and 90th percentile.
The yellow line provides a baseline case,  we predict zero for each datapoint. The dashed line shows the noise variance we use when generating the data. Note how the right figure on the first row is a zoomed-in version of the figure on the left.

On the second row, the left figure compares the true function  in orange to the predicted function in blue for one particular task. The training points and the predictions on the training points are given by the red dots and green crosses respectively. Note how these coincide since we are in the overparaamterized regime and are able to interpolate the training data.

The figure on the right shows the learned feature weights as meta training progresses. Note how all of the 500 features were equally weighted with 1 at the beginning.


In [ ]:
### start sol-d ###

### end sol-d ###


In the previous part we saw that by using a set of meta-learned feature weights that favors the true features we are able to improve our performance on the regression task during test time. Let us examine what happens if instead we use feature weights that favor the wrong set of features.
### (f)  Run the next cell and compare the performance while favoring the wrong features to using meta-learned feature weights and the all 1s feature weight.



In [ ]:
from utils_reg import test_reg, visualize_test_loss_reg
wrong_feature_weights = np.ones(params_dict['d'])
wrong_feature_weights[15:21] = 5 #We favor the wrong features
init_feature_weights = np.ones(params_dict['d'])

plt.plot(wrong_feature_weights, label = 'Wrong features favored')
plt.plot(meta_learned_feature_weights_reg_closed_form, label = 'Meta learned weights')
plt.xlabel('Feature #')
plt.ylabel('Feature weight')
plt.legend()
plt.show()

n_train_post_range, avg_test_loss, top_10_loss, bot_10_loss =  test_reg(data_dict_reg, meta_learned_feature_weights_reg_closed_form)
init_n_train_post_range, init_avg_test_loss, init_top_10_loss, init_bot_10_loss =  test_reg(data_dict_reg, init_feature_weights)

wrong_n_train_post_range, wrong_avg_test_loss, wrong_top_10_loss, wrong_bot_10_loss =  test_reg(data_dict_reg, wrong_feature_weights)


visualize_test_loss_reg(None, params_dict['n_train_inner'], n_train_post_range, avg_test_loss,
                        top_10_loss, bot_10_loss, init_n_train_post_range, init_avg_test_loss, init_top_10_loss, init_bot_10_loss, wrong_n_train_post_range=wrong_n_train_post_range,
                        wrong_avg_test_loss=wrong_avg_test_loss, wrong_top_10_loss=wrong_top_10_loss, wrong_bot_10_loss=wrong_bot_10_loss)


In [ ]:
### start sol-e ###

### end sol-e ###


## Training points on a grid

For the following cell, we change the inner loop training data spacing to grid, the spacing you worked with in the theory portions of this problem. The meta update and test data spacings remain uniform random. This means that during the inner training the features within each alias group are identical, but each feature is unique during the meta update and for the test error.

You should see the feature weights behave similarly to your limits in that the true features get favored (i.e have higher weights) but there will be a stark differnce in how certain other feature weights behave.


### (g)  Run the next two cells and answer the following questions:

1. Based on the plot of regression test loss vs n_train_post, how do the meta-learned feature weights perform compared to the case where all feature weights are 1? How do the meta-learned feature weights perform compared to the oracle (that performs regression only using the features present in the data)? Why is there a downward spike at n_train_post = 32?

2. By looking at the evolution of the feature-weights with time as we perform meta-learning can you justify the improvement in performance? In particular, can you explain why some feature weights are being sent towards zero?



In [ ]:

#Evenly spaced training points
x_type = 'grid'
params_dict = get_params_dict_reg()
cparams_dict = params_dict.copy()
cparams_dict['x_type'] = x_type
_ = meta_learning_reg_closed_form(cparams_dict)


In [ ]:
### start sol-f ###

# 1. By looking at the plot of test loss vs n_train_post we see that using the meta-learned feature
# feature weights the test performance is better than when we use the all 1s feature weights but worse than the
# oracle. As we increase n the effect of feature weights is less prominent and in all cases our test
# error goes down. There is a prominent downward spike at n_train_post = 32 since we do the training on
# inner tasks with n_train_inner = 32. The feature weights evolve during the meta learning process to downweight
# the aliases of the true feature towards 0 (when n = 32) which results in the performance being close to that
# of the oracle for this particular value of n.
# 2. We see that the weight on the favored features grows throughout the meta learning process but what is
# unique to this setting is the weights on the aliases of the true feature (corresponding to n=32) are
# getting downweighted towards 0. The weights on the other features are largely unchanged.



## Replacing closed form solution with GD

In the previous parts we saw how we could use the closed form solution to min norm least squares to train our meta learning parameter (the feature weights). However for most problems we will not have closed form solutions to the task that we wish to solve. In the absence of this, we must resort to iterative methods such as gradient descent. For the regression task, we can perform gradient descent on the squared loss. However we need to ensure that the process of computing the coefficients during inner training we preserve gradients with respect to the feature weights. Pytorch allows us to do this with the help of the higher module.

Before we test meta learning with the inner loop perfoming gradient descent instead of using the closed form solution let us check that with the right learning rate and number of iterations these two approaches lead to the same solution.


## Comparing closed form solution to GD


### (h)  Run  the cell below and play around with the different parameters and convice yourself that for sufficient number of iterations and low enough learning rate the closed form solution to min norm least squares and the result of running gradient descent starting from the all zeros vector are the same. In a previous homework you saw this analytically. For this part you don't have to answer anything.


In [ ]:
from utils import *
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import fixed
def compare_gd_closed_form(n_train, lr, num_iterations, weights, d, seed, noise_std, n_test):
    x_type = 'uniform_random'
    phi_type = 'fourier'
    np.random.seed(seed)

    k_idx = np.arange(5, 11)
    k_val = np.random.uniform(-1,1,size =len(k_idx))

    #Generate data
    x = generate_x(n_train, x_type)
    features =  featurize(x, phi_type=phi_type, d=d, normalize = True)
    y = generate_y(features, k_idx, k_val)
    y += np.random.normal(0, noise_std, y.shape)

    x_test = generate_x(n_test, 'uniform_random')
    features_test = featurize(x_test, phi_type=phi_type,d=d, normalize = True)
    y_test = generate_y(features_test, k_idx, k_val)

    closed_coeffs, closed_loss = solve_ls(features, y, weights =weights) #solve_ls
    y_test_pred_closed = features_test @ closed_coeffs


    features_t = torch.tensor(features)
    y_t = torch.tensor(y)

    #Using GD to find coeffs
    coeffs_t = torch.nn.Parameter(torch.zeros(d).double())
    optimizer = torch.optim.SGD([coeffs_t], lr=lr)
    weights_t = torch.tensor(weights)
    loss_arr = []
    for i in range(num_iterations):
        optimizer.zero_grad()
        y_pred_t = (features_t*weights_t)@coeffs_t
        loss = torch.mean((y_pred_t - y_t)**2)
        loss.backward()
        optimizer.step()
        loss_val = loss.detach().numpy()
        loss_arr.append(loss_val)


    coeffs = to_numpy(coeffs_t)
    y_pred_gd = features@coeffs
    y_test_pred_gd = features_test @ coeffs


    plt.figure(figsize = [12, 6])

    x_test = np.concatenate([x, x_test])
    y_test = np.concatenate([y, y_test])
    y_test_pred_gd = np.concatenate([y_pred_gd, y_test_pred_gd])

    y_pred_closed = features@closed_coeffs
    y_test_pred_closed = np.concatenate([y_pred_closed, y_test_pred_closed])

    idx = np.argsort(x_test)
    x_test = x_test[idx]
    y_test = y_test[idx]
    y_test_pred_gd = y_test_pred_gd[idx]
    y_test_pred_closed = y_test_pred_closed[idx]


    plt.plot(x_test, y_test, label = 'True function')
    plt.plot(x_test, y_test_pred_closed, label = 'Predicted function - closed form')
    plt.plot(x_test, y_test_pred_gd, label = 'Predicted function - gd')
    plt.legend()
    plt.show()

    #Diff in norms of coeffs
    coeffs_norm_diff = np.linalg.norm(coeffs - closed_coeffs)
    print("Coeffs norm diff: ", coeffs_norm_diff)

d = 501
noise_std = 1e-2
n_test = 1000
weights = np.ones(d)
print("Number of features: d = 501")
# compare_gd_closed_form(n_train, lr, num_iterations, weights, d)

seed_slider = generate_int_widget(desc='Seed', min_=5, val=125, max_=250)

n_train_slider = generate_int_widget(desc='Number of training points', min_=5, val=32, max_=128)
d_slider = fixed(d)
lr_slider = generate_floatlog_widget(desc='Learning rate', min_=-4, val=1e-3, max_=0, step = 0.2)
num_iterations_slider = generate_int_widget(desc='Number of iterations', min_=5, val=10, max_=5000, step = 5)
interactive_plot = interactive(compare_gd_closed_form,
                               n_train=n_train_slider,
                               lr = lr_slider,
                               num_iterations=num_iterations_slider,
                               d=d_slider, weights = fixed(weights), seed = seed_slider, noise_std = fixed(noise_std), n_test = fixed(n_test))
interactive_plot






### (i) Run the next three cells and answer the following questions:

1. With num_gd_steps = 5 does meta-learning help improve performance during test time? What happens if instead we use num_gd_steps = 1. Does meta-learning still work?

Note that in these experiments we use gradient descent in the inner loop for num_gd_steps but while testing our performance we use the closed form expression for the min norm least squares solution. This is because while testing final performance we must either run enough iterations of gradient descent to get sufficiently close to the closed form solution or use the closed form solution. However we will see that while doing the meta training even 1 gradient descent step towards the solution helps us learn the feature weights.


In [ ]:
from utils_reg import  meta_learning_reg_sgd
def get_params_dict_reg_sgd():
    params_dict = {
        #Parameters
    'seed':7,
    'n_train_inner':32, #Number of training samples for training inner task
    'n_train_meta':64, #Number of training samples for updating the meta parameter

    'n_train_post':32, #Number of training samples used after meta training has been done to learn the weights
    'n_test_post':1000, #Number of samples used for plotting and evaluating test performance after meta training has been done

    'x_type':'uniform_random', #sampling time for inner and post training tasks
    # 'x_type':'grid', #sampling time for training tasks

    'd':501, #Number of features
    'phi_type':'fourier', #Feature type
     'noise_std':1e-1, #standard deviation of awgn noise added during training
    'optimizer_type':'SGD', #Optimizer type for meta updates Can be either SGD or Adam
    'k_idx':np.arange(5,11), #Frequency range present in tasks during meta training
    'stepsize_meta':1e-2, #Stepsize used for meta updates

     'num_inner_tasks':5, #Number of inner tasks for each meta update
    'num_tasks_test':10, #Number of tasks to test on
    'num_stats': 10, #Determines how often we collect stats

    'num_iterations':100, #Iterations for training meta parameter
        'num_n_train_post_range':40, #How many points do we use to generative test loss vs n_train_post curve


    'stepsize_inner':1e-2, #Stepsize for GD update in inner tasks,
    'num_gd_steps':5, #Number of GD steps in inner task to move towards min norm ls solution

    }
    return params_dict



In [ ]:
params_dict = get_params_dict_reg_sgd()
num_gd_steps = 5
cparams_dict = params_dict.copy()
cparams_dict['num_gd_steps'] = num_gd_steps

meta_learning_reg_sgd(cparams_dict)


In [ ]:
params_dict = get_params_dict_reg_sgd()
num_gd_steps = 1
cparams_dict = params_dict.copy()
cparams_dict['num_gd_steps'] = num_gd_steps

meta_learning_reg_sgd(cparams_dict)


In [ ]:
### start sol-h ###
# We observe that for both values of num_gd_steps (1 and 5) meta learning helps us improve performance. From
# the previous cell observe that with num_gd_steps=1, the solution using gradient descent is not the same
# as the closed form solution but this does not deter the meta learning procedure. For a sanity check you can
# try running the previous cell with num_gd_steps = 0 and observe that in this case we don't learn anything.
### end  sol-h ###


## Continued...
In the next notebook `meta-learning-part2` we will see how we can use meta learning to improve performance on classification tasks. We split this problem into two notebooks for ease of running and to avoid unexpected errors caused by inconsistent states of jupyter cells.
